In [1]:
import torch
from torch.utils.data import ConcatDataset
from torchvision import transforms
import os
import numpy as np

# Import your custom utilities
from utils.dataset import LFWPairDataset, CASIAwebfaceDataset
from utils.config import DATASET_PATH
from utils.model_utils import tune_threshold, evaluate_lfw_10fold
from utils.criterion import SphereFaceNet, CosFaceNet, ArcFaceNet, CurricularFaceNet

# --- Define transforms ---
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = CASIAwebfaceDataset(
    root_dir=f'{DATASET_PATH}/CASIA-WebFace',
    split='train',
    transform=transforms.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
)
num_classes = train_dataset.num_of_identities
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using: ", device)

dataset_path = "/home/phatvo/callmePhineas/DACN/code/data_a_P"

# --- Dataset paths ---
aligned_lfw_path = f'{dataset_path}/LFW'
match_train_file = f'{dataset_path}/LFW/matchpairsDevTrain.csv'
mismatch_train_file = f'{dataset_path}/LFW/mismatchpairsDevTrain.csv'
match_test_file = f'{dataset_path}/LFW/matchpairsDevTest.csv'
mismatch_test_file = f'{dataset_path}/LFW/mismatchpairsDevTest.csv'
pairs_all_file = f'{dataset_path}/LFW/pairs.csv'

# --- Combine DevTrain + DevTest for threshold tuning ---
train_pairs_dataset_match = LFWPairDataset(
    root_dir=aligned_lfw_path,
    pairs_files=match_train_file,
    transform=test_transform
)
train_pairs_dataset_mismatch = LFWPairDataset(
    root_dir=aligned_lfw_path,
    pairs_files=mismatch_train_file,
    transform=test_transform
)
test_pairs_dataset_match = LFWPairDataset(
    root_dir=aligned_lfw_path,
    pairs_files=match_test_file,
    transform=test_transform
)
test_pairs_dataset_mismatch = LFWPairDataset(
    root_dir=aligned_lfw_path,
    pairs_files=mismatch_test_file,
    transform=test_transform
)

# Combine all pairs for threshold tuning
combined_pairs_dataset = ConcatDataset([
    train_pairs_dataset_match,
    train_pairs_dataset_mismatch,
    
    test_pairs_dataset_match,
    test_pairs_dataset_mismatch
])

/home/phatvo/miniconda3/envs/anhiuem/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/phatvo/miniconda3/envs/anhiuem/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened

Loading CASIAwebface dataset |████████████████████✗︎ (!) 439500/7400 [5939%] in 1.0s (442143.69/s) 
Using:  cuda


In [7]:
def tune_and_validate(
    model,
    batch_size=64,
):
    from contextlib import redirect_stdout
    
    log_file = "./validation_log"
    with open(log_file, 'a') as f:
        with redirect_stdout(f):
            # --- Tune threshold ---
            print("🔧 Tuning threshold on combined DevTrain + DevTest ...")
            best_threshold, best_accuracy, threshold_list, accuracy_list = tune_threshold(
                model, combined_pairs_dataset, batch_size, device, thresholds=np.arange(0.0, 0.5, 0.5)
            )
            print(f"✅ Best threshold: {best_threshold:.4f}, Combined accuracy: {best_accuracy:.2f}%")

            # --- Validate with 10-Fold LFW ---
            print("📊 Running 10-Fold cross-validation ...")
            mean_acc, std_acc = evaluate_lfw_10fold(
                model=model,
                pairs_file=pairs_all_file,
                batch_size=batch_size,
                root_dir=aligned_lfw_path,
                transform=test_transform,
                device=device,
                threshold=best_threshold
            )

            print(f"🎯 LFW 10-Fold Accuracy: {mean_acc:.2f}% ± {std_acc:.2f}% (Threshold: {best_threshold:.4f})\n\n\n")
            return best_threshold, best_accuracy, mean_acc, std_acc

# SphereFace

In [7]:
# # --- Load model ---
# model_weights_path = "./models_evaluation/CosFace_min_loss.pth"
# print(f"🔹 Loading SphereFace model from {model_weights_path}")
# model = SphereFaceNet(num_classes=num_classes).to(device)
# state_dict = torch.load(model_weights_path, map_location=device)
# model.load_state_dict(state_dict)
# model.eval()

# tune_and_validate(model=model)


# CosFace

In [8]:
# --- Load model ---
# resnet 18
model_weights_path = "./models_evaluation/CosFace_min_loss.pth"
print(f"🔹 Loading CosFace model from {model_weights_path}")
model = CosFaceNet(num_classes=num_classes).to(device)
state_dict = torch.load(model_weights_path, map_location=device)
model.load_state_dict(state_dict['model_state_dict'])
model.eval()

tune_and_validate(model=model)


🔹 Loading CosFace model from ./models_evaluation/CosFace_min_loss.pth


Evaluating (Verification):   0%|          | 0/50 [00:00<?, ?it/s]

Initialize CosFace with margin 0.35, scale 64.0


(np.float64(0.1640000000000006),
 65.84375,
 np.float64(66.21666666666667),
 np.float64(1.9805302320338358))

# ArcFace

In [ ]:
# --- Load model ---
# resnet18
model_weights_path = "./models_evaluation/ArcFace_min_loss.pth"
model_weights_path = "/home/phatvo/callmePhineas/DACN/working/result/checkpoints/ArcFace/ArcFace_min_loss.pth"
print(f"🔹 Loading ArcFace model from {model_weights_path}")
model = ArcFaceNet(num_classes=num_classes, backbone='resnet18').to(device)
state_dict = torch.load(model_weights_path, map_location=device)
model.load_state_dict(state_dict['model_state_dict'])
model.eval()

tune_and_validate(model=model)

🔹 Loading ArcFace model from /home/phatvo/callmePhineas/DACN/working/result/checkpoints/ArcFace/ArcFace_min_loss.pth


(np.float64(0.0), 50.0, np.float64(50.0), np.float64(0.0))

: 

# CurricularFace

In [10]:
# --- Load model ---
# resnet 50
model_weights_path = "./models_evaluation/CurricularFace_min_loss.pth"
print(f"🔹 Loading CurricularFace model from {model_weights_path}")
model = CurricularFaceNet(num_classes=num_classes).to(device)
state_dict = torch.load(model_weights_path, map_location=device)
model.load_state_dict(state_dict['model_state_dict'])
model.eval()

tune_and_validate(model=model)

🔹 Loading CurricularFace model from ./models_evaluation/CurricularFace_min_loss.pth


Evaluating (Verification):   0%|          | 0/50 [00:00<?, ?it/s]

Initialize CurricularFace with margin 0.5, scale 64.0, momentum 0.01


(np.float64(0.15800000000000058),
 73.65625,
 np.float64(74.63333333333333),
 np.float64(1.1874342087037928))